In [1]:
import os
import sys
import math
import time
import gzip
import random

import numpy as np
import pandas as pd
pd.set_option("display.max_columns",50)

import matplotlib.pyplot as plt
import seaborn as sns
PATH='~/Dropbox/cooperative/crystallinePolymer/data'

In [2]:
sample_list = pd.read_csv(os.path.join(PATH, 'SampleInfo.csv'), index_col=0, dtype={'CrystTemp': int, 'CrysTime': int, 'Cooling': bool})
sample_list['NucConc'] = sample_list['NucConc'].round(3)

enzyme = pd.read_csv(os.path.join(PATH, 'enzyme','test1.csv'), index_col=0)

sscurve = pd.read_csv(os.path.join(PATH, 'tensiletest','2','55d','summary.csv'))
sscurve['SampleNo']=sscurve.index+1
sscurve=sscurve.set_index('SampleNo')

In [3]:
from sklearn.ensemble import RandomForestRegressor
y_e=enzyme.query('SampleNo>16 & SampleNo<81')["day6"]
y_ss=sscurve.query('SampleNo>16 & SampleNo<81')["Yield stress/MPa"]

waxd=pd.read_csv(PATH+"/corrected_data/waxd_corrected.csv")
waxd['wQ']='w'+waxd["Q"].astype(str)
waxd_wide=waxd.pivot(index='SampleNo',columns='wQ',values='corrected_Intensity')
saxs=pd.read_csv(PATH+"/corrected_data/saxs_corrected.csv")
saxs['sQ']='s'+saxs["Q"].astype(str)
saxs_wide=saxs.pivot(index='SampleNo',columns='sQ',values='corrected_Intensity')

wcoef=waxd.filter(["SampleNo", "peak_coef","Iall_coef"]).rename(columns={'peak_coef':'wpeakcoef'})
wcoef=wcoef[~wcoef.duplicated(subset='wpeakcoef')]
scoef=saxs.filter(["SampleNo", "peak_coef"]).rename(columns={'peak_coef':'speakcoef'})
scoef=scoef[~scoef.duplicated(subset='speakcoef')]
ws_data=pd.merge(waxd_wide,saxs_wide,left_index=True,right_index=True)\
    .merge(sample_list.drop(columns="Cooling"),left_index=True,right_index=True)\
    .merge(wcoef,left_index=True,right_on= 'SampleNo')\
    .merge(scoef,left_on='SampleNo',right_on='SampleNo')

ws_data=ws_data.set_index('SampleNo')
ws_data

,w0.013803951,w0.041411851,w0.06901975,w0.096627653,w0.12423556,w0.15184346,w0.17945136,w0.20705926,w0.23466715,w0.26227507,w0.28988296,w0.31749088,w0.34509876,w0.37270665,w0.40031457,w0.42792246,w0.45553038,w0.48313826,w0.51074618,w0.5383541000000001,w0.56596196,w0.59356987,w0.62117779,w0.64878565,w0.67639357,...,s0.56083345,s0.56564748,s0.57046151,s0.57527554,s0.58008957,s0.5849036,s0.58971763,s0.59453166,s0.59934562,s0.60415965,s0.60897368,s0.61378771,s0.61860174,s0.62341577,s0.6282298000000001,s0.63304383,s0.63785779,s0.64267182,s0.64748585,CrystTemp,CrysTime,NucConc,wpeakcoef,Iall_coef,speakcoef
SampleNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.730541,1.834176,1.642493,1.486283,1.571964,1.664080,1.788581,1.579415,1.737071,1.630912,1.813075,1.572893,1.881745,1.512160,1.351116,1.367999,1.768958,1.717685,1.717192,75,5,0.000,1.017719,0.639228,0.926725
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.926494,1.018940,1.006112,1.025081,0.834238,1.255549,1.079572,0.804743,1.206884,0.847924,1.325331,1.092759,1.447713,1.286225,1.166950,1.361819,1.540004,1.172633,1.052837,75,5,0.167,1.015588,0.988414,0.949821
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.124613,0.237064,0.322569,0.030789,0.053197,0.095141,-0.049500,-0.201907,0.274799,0.458809,0.133655,0.539086,0.598287,0.073009,0.317349,0.631000,0.904763,0.300932,-0.046282,75,5,0.500,1.008572,1.167511,0.969854
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-3.515826,-3.073305,-3.253658,-3.776513,-3.800889,-3.259750,-3.657419,-3.973151,-3.355918,-3.317664,-3.173160,-2.819690,-3.190651,-3.596102,-3.515332,-2.838322,-2.463791,-3.181125,-3.247541,75,5,1.500,1.007325,1.137990,1.124905
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.734610,1.899752,1.777160,1.393095,1.631403,2.099241,1.567379,1.750069,2.194692,1.900320,1.910031,1.827088,1.941818,1.856411,1.682685,2.095693,2.290317,2.098984,1.682745,75,10,0.000,1.011012,0.917943,0.955700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-6.041136,-5.826173,-6.132209,-6.396935,-6.493184,-6.003259,-5.987103,-6.428065,-6.161246,-5.794482,-5.919016,-5.489507,-5.668442,-6.168502,-6.235857,-5.707148,-5.251950,-5.982786,-6.171265,120,20,1.500,0.986986,1.000316,1.272786
77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.779285,0.960760,0.592576,0.685238,0.681976,0.428599,0.790658,0.291657,0.260561,0.648888,0.758016,1.014670,0.990511,0.765927,0.634665,0.607473,0.549264,0.533187,0.709629,120,40,0.000,0.992105,0.939738,1.040843
78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.106624,0.254268,0.858777,0.711189,0.300761,0.434291,0.253076,0.409476,0.198331,0.053140,0.834631,1.031995,0.791058,0.663571,0.264715,0.608238,0.532211,0.332198,0.473912,120,40,0.167,0.988364,0.872975,1.046735


In [4]:
ws2e_RF = RandomForestRegressor(max_depth=30, n_estimators=500, random_state=1)
ws2e_RF.fit(ws_data,y_e)
enz_importance=pd.DataFrame({'value':ws_data.columns, 'importance': ws2e_RF.feature_importances_})
print("生分解性")
print(enz_importance.sort_values('importance',ascending=False).head(10))

ws2ss_RF = RandomForestRegressor(max_depth=30, n_estimators=500, random_state=1)
ws2ss_RF.fit(ws_data,y_ss)
ss_importance=pd.DataFrame({'value':ws_data.columns, 'importance': ws2ss_RF.feature_importances_})
print("降伏応力")
print(ss_importance.sort_values('importance',ascending=False).head(10))

生分解性
           value  importance
752   s0.1564557    0.103396
754  s0.16608374    0.079994
620   w7.2470741    0.037355
758  s0.18533982    0.030169
97    w10.698062    0.029137
767  s0.22866601    0.023255
760  s0.19496787    0.023195
770  s0.24310808    0.019081
771  s0.24792209    0.018630
773  s0.25755015    0.018387
降伏応力
                   value  importance
190           w13.265596    0.119041
617           w7.1642504    0.099902
103           w10.863709    0.067722
616  w7.1366425000000016    0.044381
189           w13.237988    0.038045
100           w10.780886    0.028011
101           w10.808494    0.017452
624           w7.3575058    0.016713
92            w10.560022    0.015448
183           w13.072341    0.015113


## 変化なし!! ##  

In [13]:
y_e.drop(24)

SampleNo
17    0.093531
18    0.128673
19    0.101083
20    0.126852
21    0.083261
        ...   
76    0.037866
77    0.021729
78    0.026843
79    0.027511
80    0.009505
Name: day6, Length: 63, dtype: float64

In [15]:
##  SAXSが極端に負になっているSampleNo 24を除去したら？
sample_list.loc[24]

ws2e_RF_ex = RandomForestRegressor(max_depth=30, n_estimators=500, random_state=1)
ws2e_RF_ex.fit(ws_data.drop(24),y_e.drop(24))
enz_importance_ex=pd.DataFrame({'value':ws_data.drop(24).columns, 'importance': ws2e_RF_ex.feature_importances_})
print("生分解性")
print(enz_importance_ex.sort_values('importance',ascending=False).head(10))
ws2ss_RF_ex = RandomForestRegressor(max_depth=30, n_estimators=500, random_state=1)
ws2ss_RF_ex.fit(ws_data.drop(24),y_ss.drop(24))
ss_importance_ex=pd.DataFrame({'value':ws_data.drop(24).columns, 'importance': ws2ss_RF_ex.feature_importances_})
print("降伏応力")
print(ss_importance_ex.sort_values('importance',ascending=False).head(10))

生分解性
                  value  importance
752          s0.1564557    0.189625
754         s0.16608374    0.033869
749         s0.14201362    0.030114
767         s0.22866601    0.023946
758         s0.18533982    0.022773
751         s0.15164167    0.022625
630  w7.523152799999999    0.022149
745         s0.12275754    0.020506
770         s0.24310808    0.018815
90           w10.504807    0.018063
降伏応力
                   value  importance
617           w7.1642504    0.148758
190           w13.265596    0.103960
100           w10.780886    0.044162
616  w7.1366425000000016    0.041422
103           w10.863709    0.036218
189           w13.237988    0.019759
860            speakcoef    0.017911
104           w10.891317    0.016853
92            w10.560022    0.016389
94            w10.615238    0.015709



## 補正前(rawdata)では？ ##

In [7]:
waxd_raw=pd.read_csv(PATH+"/corrected_data/waxd_rawdata.csv", index_col=0)
waxd_raw['wQ']='w'+waxd_raw["Q"].astype(str)
waxd_wide_raw=waxd_raw.pivot(index='SampleNo',columns='wQ',values='Intensity')
saxs_raw=pd.read_csv(PATH+"/corrected_data/saxs_rawdata.csv", index_col=0)
saxs_raw['sQ']='s'+saxs_raw["Q"].astype(str)
saxs_wide_raw=saxs_raw.pivot(index='SampleNo',columns='sQ',values='Intensity')
ws_data_raw=pd.merge(waxd_wide_raw,saxs_wide_raw,left_index=True,right_index=True)\
    .merge(sample_list.drop(columns="Cooling"),left_index=True,right_index=True)



ws2e_RF_raw = RandomForestRegressor(max_depth=30, n_estimators=500, random_state=1)
ws2e_RF_raw.fit(ws_data_raw,y_e)

e_importance_raw=pd.DataFrame({'value':ws_data_raw.columns, 'importance': ws2e_RF_raw.feature_importances_})
print("生分解性と")
print(e_importance_raw.sort_values('importance',ascending=False).head(10))

ws2ss_RF_raw = RandomForestRegressor(max_depth=30, n_estimators=500, random_state=1)
ws2ss_RF_raw.fit(ws_data_raw,y_ss)

ss_importance_raw=pd.DataFrame({'value':ws_data_raw.columns, 'importance': ws2ss_RF_raw.feature_importances_})
print("降伏応力と")
print(ss_importance_raw.sort_values('importance',ascending=False).head(10))

生分解性と
                   value  importance
855            CrystTemp    0.116152
460           w2.8298099    0.088295
803          s0.40197077    0.051041
802          s0.39715675    0.043715
457  w2.7469862000000003    0.042982
459            w2.802202    0.034439
482  w3.4371836000000004    0.021432
815          s0.45973903    0.020842
812          s0.44529697    0.018870
119           w11.305435    0.017849
降伏応力と
                   value  importance
750          s0.14682765    0.066520
751          s0.15164167    0.055171
564  w5.7010317000000015    0.039026
565   w5.728639599999998    0.036318
481  w3.4095757000000004    0.034807
754          s0.16608374    0.031459
480           w3.3819678    0.025558
125           w11.471083    0.022639
124           w11.443475    0.015689
753          s0.16126971    0.014696
